In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
%matplotlib inline

Using TensorFlow backend.


In [2]:
df = pd.read_csv('train.csv').drop(columns=['Id'])
def data():
    X_train, X_val, y_train, y_val = train_test_split(MinMaxScaler().fit_transform(df.iloc[:,1:]).reshape((-1, 28, 28, 1)), to_categorical(df.iloc[:,0], num_classes=5), test_size=0.2)
    return X_train, y_train, X_val, y_val

def model(X_train, y_train, X_val, y_val):
    model = Sequential()
    model_choice = {{choice(['one', 'two'])}}
    if model_choice == 'one':
        model.add(Conv2D(16, kernel_size=3, activation='relu',padding='same', input_shape=(28,28,1)))
        model.add(Conv2D(16, kernel_size=3, activation='relu',padding='same'))
        model.add(MaxPool2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
        model.add(Conv2D(32, kernel_size=3, activation='relu'))
        model.add(Conv2D(32, kernel_size=3, activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPool2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
    elif model_choice == 'two':
        model.add(Conv2D(32, kernel_size=3, activation='relu',padding='same', input_shape=(28,28,1)))
        model.add(Conv2D(32, kernel_size=3, activation='relu',padding='same'))
        model.add(MaxPool2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
        model.add(Conv2D(64, kernel_size=3, activation='relu'))
        model.add(Conv2D(64, kernel_size=3, activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPool2D(pool_size=2,strides=2))
        model.add(Dropout({{uniform(0, 1)}}))
    
    model.add(Flatten())
    model.add(Dense({{choice([256, 512,1024])}}, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout({{uniform(0, 1)}}))
    choiceval = {{choice(['one', 'two'])}}
    if choiceval == 'two':
        model.add(Dense({{choice([256, 512,1024])}}, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout({{uniform(0, 1)}}))
    
    model.add(Dense(5, activation='softmax'))
    
    adam = Adam(lr=0.001)
    
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)
    model.fit(X_train, y_train, batch_size=256, epochs=15, verbose=2, validation_data=(X_val, y_val))
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Val accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

def acc_plot(history, name):
    plt.figure(figsize=(10,6))
    plt.plot(history['accuracy'], label='training accuracy')
    plt.plot(history['loss'], label='training loss')    
    plt.plot(history['val_accuracy'], label='validation accuracy')
    plt.plot(history['val_loss'], label='validation loss')
    plt.title(name + ' Accuracy and Loss')
    plt.xlabel('epochs')
    plt.legend()
#     plt.savefig(name +'_Result.png', bbox_inches='tight', dpi=300)
    return plt.show()

def gen_csv(model, name):
    result = dfTest[['Id']].copy()
    result['Label'] = model.predict_classes(MinMaxScaler().fit_transform(dfTest.iloc[:,1:]).reshape((-1, 28, 28, 1)))
    result.to_csv('result' + name + '.csv', encoding='utf-8', index=False)

In [9]:
X_train, y_train, X_val, y_val = data()
best_run, best_model = optim.minimize(model=model, data=data, algo=tpe.suggest, max_evals=50, trials=Trials(), notebook_name='tuning')

>>> Imports:
#coding=utf-8

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.preprocessing import MinMaxScaler
except:
    pass

try:
    from tensorflow.keras import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
except:
    pass

try:
    from tensorflow.keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop
except:
    pass

try:
    from tensorflow.keras.utils import to_categorical, plot_model
except:
    pass

try:
    from tensorflow.keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    

48000/48000 - 2s - loss: 0.9235 - accuracy: 0.5912 - val_loss: 3.7055 - val_accuracy: 0.1987

Epoch 6/15                                                                       
48000/48000 - 2s - loss: 0.8922 - accuracy: 0.6084 - val_loss: 3.8343 - val_accuracy: 0.1985

Epoch 7/15                                                                       
48000/48000 - 2s - loss: 0.8741 - accuracy: 0.6165 - val_loss: 4.1887 - val_accuracy: 0.1984

Epoch 8/15                                                                       
48000/48000 - 2s - loss: 0.8543 - accuracy: 0.6280 - val_loss: 4.1929 - val_accuracy: 0.1984

Epoch 9/15                                                                       
48000/48000 - 2s - loss: 0.8442 - accuracy: 0.6335 - val_loss: 4.6069 - val_accuracy: 0.1982

Epoch 10/15                                                                      
48000/48000 - 2s - loss: 0.8305 - accuracy: 0.6381 - val_loss: 5.1422 - val_accuracy: 0.1982

Epoch 11/15               

Val accuracy:                                                                    
0.23291667                                                                       
Train on 48000 samples, validate on 12000 samples                                 
Epoch 1/15                                                                        
48000/48000 - 3s - loss: 0.8633 - accuracy: 0.6622 - val_loss: 2.0681 - val_accuracy: 0.3624

Epoch 2/15                                                                        
48000/48000 - 3s - loss: 0.5439 - accuracy: 0.7818 - val_loss: 2.0080 - val_accuracy: 0.4529

Epoch 3/15                                                                        
48000/48000 - 2s - loss: 0.4836 - accuracy: 0.8062 - val_loss: 2.2241 - val_accuracy: 0.4547

Epoch 4/15                                                                        
48000/48000 - 2s - loss: 0.4527 - accuracy: 0.8167 - val_loss: 2.9170 - val_accuracy: 0.4250

Epoch 5/15                                   

Epoch 10/15                                                                       
48000/48000 - 2s - loss: 0.5301 - accuracy: 0.7894 - val_loss: 0.5939 - val_accuracy: 0.7456

Epoch 11/15                                                                       
48000/48000 - 3s - loss: 0.5137 - accuracy: 0.7988 - val_loss: 0.7975 - val_accuracy: 0.7036

Epoch 12/15                                                                       
48000/48000 - 2s - loss: 0.5033 - accuracy: 0.7996 - val_loss: 0.6532 - val_accuracy: 0.7383

Epoch 13/15                                                                       
48000/48000 - 3s - loss: 0.4939 - accuracy: 0.8029 - val_loss: 0.5284 - val_accuracy: 0.7741

Epoch 14/15                                                                       
48000/48000 - 2s - loss: 0.4821 - accuracy: 0.8092 - val_loss: 0.5072 - val_accuracy: 0.7725

Epoch 15/15                                                                       
48000/48000 - 2s - loss: 0.4801 

Epoch 4/15                                                                        
48000/48000 - 2s - loss: 1.2712 - accuracy: 0.4602 - val_loss: 1.6247 - val_accuracy: 0.3305

Epoch 5/15                                                                        
48000/48000 - 2s - loss: 1.2131 - accuracy: 0.4797 - val_loss: 1.0702 - val_accuracy: 0.5118

Epoch 6/15                                                                        
48000/48000 - 2s - loss: 1.1444 - accuracy: 0.5058 - val_loss: 1.1204 - val_accuracy: 0.4878

Epoch 7/15                                                                        
48000/48000 - 2s - loss: 1.1050 - accuracy: 0.5199 - val_loss: 1.5113 - val_accuracy: 0.3685

Epoch 8/15                                                                        
48000/48000 - 2s - loss: 1.0678 - accuracy: 0.5319 - val_loss: 0.6928 - val_accuracy: 0.7362

Epoch 9/15                                                                        
48000/48000 - 2s - loss: 1.0301 

48000/48000 - 3s - loss: 0.5132 - accuracy: 0.7922 - val_loss: 0.4849 - val_accuracy: 0.7901

Epoch 15/15                                                                       
48000/48000 - 2s - loss: 0.5036 - accuracy: 0.7958 - val_loss: 0.6172 - val_accuracy: 0.7412

Val accuracy:                                                                     
0.74125                                                                           
Train on 48000 samples, validate on 12000 samples                                 
Epoch 1/15                                                                        
48000/48000 - 3s - loss: 1.2969 - accuracy: 0.5299 - val_loss: 1.9120 - val_accuracy: 0.2080

Epoch 2/15                                                                        
48000/48000 - 2s - loss: 0.7664 - accuracy: 0.6801 - val_loss: 1.3844 - val_accuracy: 0.4451

Epoch 3/15                                                                        
48000/48000 - 2s - loss: 0.6622 - accuracy:

48000/48000 - 2s - loss: 0.3666 - accuracy: 0.8500 - val_loss: 1.3639 - val_accuracy: 0.5715

Epoch 9/15                                                                        
48000/48000 - 2s - loss: 0.3654 - accuracy: 0.8509 - val_loss: 0.6378 - val_accuracy: 0.7408

Epoch 10/15                                                                       
48000/48000 - 2s - loss: 0.3593 - accuracy: 0.8536 - val_loss: 0.6016 - val_accuracy: 0.7628

Epoch 11/15                                                                       
48000/48000 - 2s - loss: 0.3453 - accuracy: 0.8584 - val_loss: 0.4687 - val_accuracy: 0.8121

Epoch 12/15                                                                       
48000/48000 - 2s - loss: 0.3364 - accuracy: 0.8627 - val_loss: 0.8046 - val_accuracy: 0.7168

Epoch 13/15                                                                       
48000/48000 - 2s - loss: 0.3313 - accuracy: 0.8644 - val_loss: 0.4345 - val_accuracy: 0.8212

Epoch 14/15          

48000/48000 - 2s - loss: 0.5357 - accuracy: 0.7831 - val_loss: 0.9208 - val_accuracy: 0.5904

Epoch 3/15                                                                        
48000/48000 - 2s - loss: 0.4771 - accuracy: 0.8088 - val_loss: 0.4179 - val_accuracy: 0.8372

Epoch 4/15                                                                        
48000/48000 - 2s - loss: 0.4415 - accuracy: 0.8200 - val_loss: 0.5437 - val_accuracy: 0.7731

Epoch 5/15                                                                        
48000/48000 - 2s - loss: 0.4206 - accuracy: 0.8299 - val_loss: 0.3901 - val_accuracy: 0.8392

Epoch 6/15                                                                        
48000/48000 - 2s - loss: 0.4043 - accuracy: 0.8368 - val_loss: 0.4338 - val_accuracy: 0.8165

Epoch 7/15                                                                        
48000/48000 - 2s - loss: 0.3937 - accuracy: 0.8402 - val_loss: 0.4041 - val_accuracy: 0.8332

Epoch 8/15           

Val accuracy:                                                                     
0.7690833                                                                         
Train on 48000 samples, validate on 12000 samples                                 
Epoch 1/15                                                                        
48000/48000 - 3s - loss: 0.7948 - accuracy: 0.6788 - val_loss: 1.7242 - val_accuracy: 0.2003

Epoch 2/15                                                                        
48000/48000 - 2s - loss: 0.5521 - accuracy: 0.7748 - val_loss: 0.9980 - val_accuracy: 0.5512

Epoch 3/15                                                                        
48000/48000 - 2s - loss: 0.4923 - accuracy: 0.8001 - val_loss: 0.5925 - val_accuracy: 0.7459

Epoch 4/15                                                                        
48000/48000 - 2s - loss: 0.4601 - accuracy: 0.8140 - val_loss: 0.3977 - val_accuracy: 0.8396

Epoch 5/15                                 

Epoch 10/15                                                                       
48000/48000 - 2s - loss: 0.4339 - accuracy: 0.8264 - val_loss: 0.3617 - val_accuracy: 0.8468

Epoch 11/15                                                                       
48000/48000 - 2s - loss: 0.4135 - accuracy: 0.8342 - val_loss: 0.3532 - val_accuracy: 0.8491

Epoch 12/15                                                                       
48000/48000 - 2s - loss: 0.4094 - accuracy: 0.8366 - val_loss: 0.4228 - val_accuracy: 0.8167

Epoch 13/15                                                                       
48000/48000 - 2s - loss: 0.4011 - accuracy: 0.8404 - val_loss: 0.3889 - val_accuracy: 0.8363

Epoch 14/15                                                                       
48000/48000 - 2s - loss: 0.3933 - accuracy: 0.8438 - val_loss: 0.3445 - val_accuracy: 0.8572

Epoch 15/15                                                                       
48000/48000 - 2s - loss: 0.3834 

In [6]:
print(best_run)

{'Dense': 0, 'Dense_1': 1, 'Dropout': 0.5918865882659488, 'Dropout_1': 0.8876225621897421, 'Dropout_2': 0.132436780822, 'Dropout_3': 0.2576750244735247, 'Dropout_4': 0.3144048645633753, 'Dropout_5': 0.12715375214014452, 'model_choice': 1, 'model_choice_1': 1}


In [8]:
print(best_model.evaluate(X_val, y_val))

12000/12000 [==============================] - 1s 118us/sample - loss: 0.2444 - accuracy: 0.9014
[0.244437867273887, 0.90141666]


In [27]:
model = Sequential([
    Conv2D(32, kernel_size=3, activation='relu',padding='same', input_shape=(28,28,1)),
    Conv2D(32, kernel_size=3, activation='relu',padding='same'),
    MaxPool2D(pool_size=2,strides=2),
    Dropout(0.13243678),
    Conv2D(64, kernel_size=3, activation='relu'),
    Conv2D(64, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPool2D(pool_size=2,strides=2),
    Dropout(0.25767502),
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.31440486),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.12715375),
    Dense(5, activation='softmax')
])
X_train, y_train, X_val, y_val = data()
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(lr=0.001))
model.fit(X_train, y_train, batch_size=128, epochs=40, verbose=2, validation_data=(X_val, y_val))
score, acc = model.evaluate(X_val, y_val, verbose=0)
print('Val accuracy:', acc)

Train on 48000 samples, validate on 12000 samples
Epoch 1/40
48000/48000 - 5s - loss: 0.6241 - accuracy: 0.7536 - val_loss: 0.9170 - val_accuracy: 0.6100
Epoch 2/40
48000/48000 - 4s - loss: 0.4502 - accuracy: 0.8207 - val_loss: 0.4001 - val_accuracy: 0.8428
Epoch 3/40
48000/48000 - 4s - loss: 0.4085 - accuracy: 0.8361 - val_loss: 0.3881 - val_accuracy: 0.8460
Epoch 4/40
48000/48000 - 4s - loss: 0.3833 - accuracy: 0.8436 - val_loss: 1.8247 - val_accuracy: 0.5753
Epoch 5/40
48000/48000 - 4s - loss: 0.3713 - accuracy: 0.8494 - val_loss: 0.4946 - val_accuracy: 0.7927
Epoch 6/40
48000/48000 - 4s - loss: 0.3487 - accuracy: 0.8588 - val_loss: 0.8991 - val_accuracy: 0.7042
Epoch 7/40
48000/48000 - 4s - loss: 0.3365 - accuracy: 0.8629 - val_loss: 0.2905 - val_accuracy: 0.8832
Epoch 8/40
48000/48000 - 4s - loss: 0.3263 - accuracy: 0.8689 - val_loss: 0.8465 - val_accuracy: 0.6973
Epoch 9/40
48000/48000 - 4s - loss: 0.3129 - accuracy: 0.8717 - val_loss: 0.4221 - val_accuracy: 0.8282
Epoch 10/40
48

In [33]:
def modelCNN1(X_train, y_train, X_val, y_val):
    CNN1 = Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)),
        MaxPool2D(pool_size=(2, 2)),
        Dropout({{uniform(0, 1)}}),
        Flatten(),
        Dense({{choice([256, 512,1024])}}, activation='relu'),
        Dense(5, activation='softmax')
    ])

    choiceval = {{choice(['adam', 'sgd', 'rmsprop'])}}
    if choiceval == 'adam':
        optim = Adam(lr={{choice([10**-3, 10**-2, 10**-1])}})
    elif choiceval == 'rmsprop':
        optim = RMSprop(lr={{choice([10**-3, 10**-2, 10**-1])}})
    else:
        optim = SGD(lr={{choice([10**-3, 10**-2, 10**-1])}})
    CNN1.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=optim)
    CNN1.fit(X_train, y_train, batch_size=256, epochs=15, verbose=2, validation_data=(X_val, y_val))
    score, acc = CNN1.evaluate(X_val, y_val, verbose=0)
    print('Val accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': CNN1}

X_train, y_train, X_val, y_val = data()
best_run, best_model = optim.minimize(model=modelCNN1, data=data, algo=tpe.suggest, max_evals=30, trials=Trials(), notebook_name='tuning')

>>> Imports:
#coding=utf-8

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.preprocessing import MinMaxScaler
except:
    pass

try:
    from tensorflow.keras import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
except:
    pass

try:
    from tensorflow.keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop
except:
    pass

try:
    from tensorflow.keras.utils import to_categorical, plot_model
except:
    pass

try:
    from tensorflow.keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    

48000/48000 - 1s - loss: 1.0218 - accuracy: 0.5707 - val_loss: 0.9919 - val_accuracy: 0.5787

Epoch 15/15                                                                      
48000/48000 - 1s - loss: 0.9913 - accuracy: 0.5799 - val_loss: 0.9635 - val_accuracy: 0.5871

Val accuracy:                                                                    
0.58708334                                                                       
Train on 48000 samples, validate on 12000 samples                                
Epoch 1/15                                                                       
48000/48000 - 2s - loss: 227.8601 - accuracy: 0.3668 - val_loss: 1.6278 - val_accuracy: 0.1982

Epoch 2/15                                                                       
48000/48000 - 1s - loss: 1.6132 - accuracy: 0.2006 - val_loss: 1.6170 - val_accuracy: 0.1970

Epoch 3/15                                                                       
48000/48000 - 1s - loss: 1.6131 - accuracy: 0.20

48000/48000 - 1s - loss: 1.6120 - accuracy: 0.1983 - val_loss: 1.6101 - val_accuracy: 0.1970

Epoch 9/15                                                                        
48000/48000 - 1s - loss: 1.6115 - accuracy: 0.1990 - val_loss: 1.6142 - val_accuracy: 0.1972

Epoch 10/15                                                                       
48000/48000 - 1s - loss: 1.6120 - accuracy: 0.1980 - val_loss: 1.6149 - val_accuracy: 0.1972

Epoch 11/15                                                                       
48000/48000 - 1s - loss: 1.6118 - accuracy: 0.2027 - val_loss: 1.6106 - val_accuracy: 0.2055

Epoch 12/15                                                                       
48000/48000 - 1s - loss: 1.6117 - accuracy: 0.2022 - val_loss: 1.6118 - val_accuracy: 0.1972

Epoch 13/15                                                                       
48000/48000 - 1s - loss: 1.6115 - accuracy: 0.1997 - val_loss: 1.6117 - val_accuracy: 0.1972

Epoch 14/15          

48000/48000 - 1s - loss: 1.0500 - accuracy: 0.5540 - val_loss: 0.9141 - val_accuracy: 0.5978

Epoch 3/15                                                                        
48000/48000 - 1s - loss: 0.8724 - accuracy: 0.6238 - val_loss: 0.7989 - val_accuracy: 0.6489

Epoch 4/15                                                                        
48000/48000 - 1s - loss: 0.8039 - accuracy: 0.6479 - val_loss: 0.7598 - val_accuracy: 0.6651

Epoch 5/15                                                                        
48000/48000 - 1s - loss: 0.7675 - accuracy: 0.6691 - val_loss: 0.7224 - val_accuracy: 0.6936

Epoch 6/15                                                                        
48000/48000 - 1s - loss: 0.7433 - accuracy: 0.6818 - val_loss: 0.6989 - val_accuracy: 0.7044

Epoch 7/15                                                                        
48000/48000 - 1s - loss: 0.7182 - accuracy: 0.6951 - val_loss: 0.6694 - val_accuracy: 0.7199

Epoch 8/15           

Val accuracy:                                                                     
0.56458336                                                                        
Train on 48000 samples, validate on 12000 samples                                 
Epoch 1/15                                                                        
48000/48000 - 2s - loss: 0.6830 - accuracy: 0.7068 - val_loss: 0.4890 - val_accuracy: 0.8037

Epoch 2/15                                                                        
48000/48000 - 1s - loss: 0.4769 - accuracy: 0.8052 - val_loss: 0.3962 - val_accuracy: 0.8433

Epoch 3/15                                                                        
48000/48000 - 1s - loss: 0.4204 - accuracy: 0.8288 - val_loss: 0.3814 - val_accuracy: 0.8486

Epoch 4/15                                                                        
48000/48000 - 1s - loss: 0.3869 - accuracy: 0.8436 - val_loss: 0.3353 - val_accuracy: 0.8673

Epoch 5/15                                 

Epoch 10/15                                                                       
48000/48000 - 1s - loss: 0.2921 - accuracy: 0.8834 - val_loss: 0.2959 - val_accuracy: 0.8838

Epoch 11/15                                                                       
48000/48000 - 1s - loss: 0.2818 - accuracy: 0.8854 - val_loss: 0.2981 - val_accuracy: 0.8775

Epoch 12/15                                                                       
48000/48000 - 1s - loss: 0.2702 - accuracy: 0.8923 - val_loss: 0.2931 - val_accuracy: 0.8827

Epoch 13/15                                                                       
48000/48000 - 1s - loss: 0.2597 - accuracy: 0.8956 - val_loss: 0.2824 - val_accuracy: 0.8844

Epoch 14/15                                                                       
48000/48000 - 1s - loss: 0.2531 - accuracy: 0.8974 - val_loss: 0.2791 - val_accuracy: 0.8897

Epoch 15/15                                                                       
48000/48000 - 1s - loss: 0.2480 

In [32]:
print(best_run)
print(best_model.evaluate(X_val, y_val))

{'Dense': 0, 'Dropout': 0.4261775040222384, 'choiceval': 0, 'lr': 0, 'lr_1': 0, 'lr_2': 2}
12000/12000 [==============================] - 1s 84us/sample - loss: 0.2697 - accuracy: 0.8941
[0.2697447367807229, 0.8940833]


In [34]:
print(best_run)
print(best_model.evaluate(X_val, y_val))

{'Dense': 1, 'Dropout': 0.5415092931501145, 'choiceval': 0, 'lr': 0, 'lr_1': 2, 'lr_2': 1}
12000/12000 [==============================] - 1s 77us/sample - loss: 0.2625 - accuracy: 0.8942
[0.26247730090220767, 0.89416665]


In [35]:
def modelCNN2(X_train, y_train, X_val, y_val):
    CNN2 = Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28,28,1)),
        BatchNormalization(),
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Dropout({{uniform(0, 1)}}),
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
        MaxPool2D(pool_size=(2, 2)),
        Dropout({{uniform(0, 1)}}),
        Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Dropout({{uniform(0, 1)}}),
        Flatten(),
        Dense({{choice([256, 512,1024])}}, activation='relu'),
        BatchNormalization(),
        Dropout({{uniform(0, 1)}}),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout({{uniform(0, 1)}}),
        Dense(5, activation='softmax')
    ])

    choiceval = {{choice(['adam', 'sgd', 'rmsprop'])}}
    if choiceval == 'adam':
        optim = Adam(lr={{choice([10**-3, 10**-2, 10**-1])}})
    elif choiceval == 'rmsprop':
        optim = RMSprop(lr={{choice([10**-3, 10**-2, 10**-1])}})
    else:
        optim = SGD(lr={{choice([10**-3, 10**-2, 10**-1])}})
    CNN2.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=optim)
    CNN2.fit(X_train, y_train, batch_size=256, epochs=15, verbose=2, validation_data=(X_val, y_val))
    score, acc = CNN2.evaluate(X_val, y_val, verbose=0)
    print('Val accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': CNN2}

X_train, y_train, X_val, y_val = data()
best_run, best_model = optim.minimize(model=modelCNN2, data=data, algo=tpe.suggest, max_evals=30, trials=Trials(), notebook_name='tuning')

>>> Imports:
#coding=utf-8

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.preprocessing import MinMaxScaler
except:
    pass

try:
    from tensorflow.keras import Sequential
except:
    pass

try:
    from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
except:
    pass

try:
    from tensorflow.keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop
except:
    pass

try:
    from tensorflow.keras.utils import to_categorical, plot_model
except:
    pass

try:
    from tensorflow.keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    

48000/48000 - 5s - loss: 1.6908 - accuracy: 0.2958 - val_loss: 2.5094 - val_accuracy: 0.2020

Epoch 10/15                                                                      
48000/48000 - 5s - loss: 1.6514 - accuracy: 0.3078 - val_loss: 2.5355 - val_accuracy: 0.2020

Epoch 11/15                                                                      
48000/48000 - 5s - loss: 1.6205 - accuracy: 0.3111 - val_loss: 2.5535 - val_accuracy: 0.2020

Epoch 12/15                                                                      
48000/48000 - 5s - loss: 1.5939 - accuracy: 0.3165 - val_loss: 2.5789 - val_accuracy: 0.2020

Epoch 13/15                                                                      
48000/48000 - 5s - loss: 1.5581 - accuracy: 0.3260 - val_loss: 2.6050 - val_accuracy: 0.2052

Epoch 14/15                                                                      
48000/48000 - 5s - loss: 1.5354 - accuracy: 0.3284 - val_loss: 2.6297 - val_accuracy: 0.2628

Epoch 15/15               

48000/48000 - 5s - loss: 0.6133 - accuracy: 0.7538 - val_loss: 1.8052 - val_accuracy: 0.4911

Epoch 4/15                                                                        
48000/48000 - 5s - loss: 0.5541 - accuracy: 0.7805 - val_loss: 0.5018 - val_accuracy: 0.7937

Epoch 5/15                                                                        
48000/48000 - 5s - loss: 0.5365 - accuracy: 0.7885 - val_loss: 1.2857 - val_accuracy: 0.5950

Epoch 6/15                                                                        
48000/48000 - 5s - loss: 0.4971 - accuracy: 0.8022 - val_loss: 0.6104 - val_accuracy: 0.7632

Epoch 7/15                                                                        
48000/48000 - 5s - loss: 0.5249 - accuracy: 0.7896 - val_loss: 0.5354 - val_accuracy: 0.7830

Epoch 8/15                                                                        
48000/48000 - 5s - loss: 0.4813 - accuracy: 0.8084 - val_loss: 1.2972 - val_accuracy: 0.5692

Epoch 9/15           

48000/48000 - 5s - loss: 0.7987 - accuracy: 0.6719 - val_loss: 2.1040 - val_accuracy: 0.3137

Epoch 14/15                                                                       
48000/48000 - 5s - loss: 0.7900 - accuracy: 0.6759 - val_loss: 1.3973 - val_accuracy: 0.4207

Epoch 15/15                                                                       
48000/48000 - 5s - loss: 0.7780 - accuracy: 0.6811 - val_loss: 2.1445 - val_accuracy: 0.3132

Val accuracy:                                                                     
0.31316668                                                                        
Train on 48000 samples, validate on 12000 samples                                 
Epoch 1/15                                                                        
48000/48000 - 8s - loss: 1.4120 - accuracy: 0.4466 - val_loss: 3.5174 - val_accuracy: 0.1982

Epoch 2/15                                                                        
48000/48000 - 5s - loss: 0.9677 - accuracy:

48000/48000 - 5s - loss: 0.2936 - accuracy: 0.8898 - val_loss: 0.4043 - val_accuracy: 0.8511

Epoch 8/15                                                                        
48000/48000 - 5s - loss: 0.2786 - accuracy: 0.8947 - val_loss: 0.3414 - val_accuracy: 0.8700

Epoch 9/15                                                                        
48000/48000 - 5s - loss: 0.2608 - accuracy: 0.9030 - val_loss: 0.3834 - val_accuracy: 0.8649

Epoch 10/15                                                                       
48000/48000 - 5s - loss: 0.2496 - accuracy: 0.9071 - val_loss: 0.8397 - val_accuracy: 0.7552

Epoch 11/15                                                                       
48000/48000 - 5s - loss: 0.2282 - accuracy: 0.9153 - val_loss: 0.3261 - val_accuracy: 0.8873

Epoch 12/15                                                                       
48000/48000 - 5s - loss: 0.2217 - accuracy: 0.9179 - val_loss: 0.3374 - val_accuracy: 0.8837

Epoch 13/15          

48000/48000 - 7s - loss: 0.8945 - accuracy: 0.6626 - val_loss: 26.1270 - val_accuracy: 0.2107

Epoch 2/15                                                                        
48000/48000 - 5s - loss: 0.5670 - accuracy: 0.7831 - val_loss: 17.0442 - val_accuracy: 0.2459

Epoch 3/15                                                                        
48000/48000 - 5s - loss: 0.4853 - accuracy: 0.8178 - val_loss: 1.6845 - val_accuracy: 0.5968

Epoch 4/15                                                                        
48000/48000 - 5s - loss: 0.4363 - accuracy: 0.8387 - val_loss: 1.7037 - val_accuracy: 0.6302

Epoch 5/15                                                                        
48000/48000 - 5s - loss: 0.4101 - accuracy: 0.8499 - val_loss: 1.7178 - val_accuracy: 0.6544

Epoch 6/15                                                                        
48000/48000 - 5s - loss: 0.3775 - accuracy: 0.8602 - val_loss: 0.8114 - val_accuracy: 0.7558

Epoch 7/15         

In [36]:
print(best_run)
print(best_model.evaluate(X_val, y_val))

{'Dense': 0, 'Dropout': 0.001154900332688294, 'Dropout_1': 0.014244592769077968, 'Dropout_2': 0.1339600303017593, 'Dropout_3': 0.25215126323219333, 'Dropout_4': 0.7446448685656921, 'choiceval': 2, 'lr': 0, 'lr_1': 1, 'lr_2': 2}
12000/12000 [==============================] - 1s 112us/sample - loss: 0.3544 - accuracy: 0.8885
[0.3543927517036597, 0.8885]


In [38]:
CNN2 = Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28,28,1)),
    BatchNormalization(),
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.0011549),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.01424459),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.13396003),
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.25215126),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.74464487),
    Dense(5, activation='softmax')
])
CNN2.compile(optimizer=RMSprop(lr=0.1), loss="categorical_crossentropy", metrics=["accuracy"])
historyCNN2 = CNN2.fit(X_train, y_train, epochs = 80, batch_size=128, validation_data = (X_val, y_val))

Train on 48000 samples, validate on 12000 samples
Epoch 1/80
48000/48000 [==============================] - 8s 161us/sample - loss: 1.3309 - accuracy: 0.5417 - val_loss: 9.0803 - val_accuracy: 0.3848
Epoch 2/80
48000/48000 [==============================] - 6s 129us/sample - loss: 0.9961 - accuracy: 0.6558 - val_loss: 1.2276 - val_accuracy: 0.7499
Epoch 3/80
48000/48000 [==============================] - 6s 128us/sample - loss: 0.9140 - accuracy: 0.6917 - val_loss: 0.7839 - val_accuracy: 0.7531
Epoch 4/80
48000/48000 [==============================] - 6s 130us/sample - loss: 0.8679 - accuracy: 0.7082 - val_loss: 1.2218 - val_accuracy: 0.7332
Epoch 5/80
48000/48000 [==============================] - 6s 127us/sample - loss: 0.8346 - accuracy: 0.7293 - val_loss: 42.5674 - val_accuracy: 0.7084
Epoch 6/80
48000/48000 [==============================] - 6s 129us/sample - loss: 0.8076 - accuracy: 0.7376 - val_loss: 14.8683 - val_accuracy: 0.6648
Epoch 7/80
48000/48000 [========================

Epoch 55/80
48000/48000 [==============================] - 6s 128us/sample - loss: 0.4369 - accuracy: 0.9065 - val_loss: 0.8344 - val_accuracy: 0.8607
Epoch 56/80
48000/48000 [==============================] - 6s 127us/sample - loss: 0.4094 - accuracy: 0.9106 - val_loss: 2.0273 - val_accuracy: 0.8519
Epoch 57/80
48000/48000 [==============================] - 6s 130us/sample - loss: 0.4104 - accuracy: 0.9135 - val_loss: 0.9367 - val_accuracy: 0.8461
Epoch 58/80
48000/48000 [==============================] - 6s 127us/sample - loss: 0.4327 - accuracy: 0.9102 - val_loss: 0.8036 - val_accuracy: 0.8582
Epoch 59/80
48000/48000 [==============================] - 6s 127us/sample - loss: 0.4256 - accuracy: 0.9091 - val_loss: 1.0851 - val_accuracy: 0.8392
Epoch 60/80
48000/48000 [==============================] - 6s 129us/sample - loss: 0.3993 - accuracy: 0.9148 - val_loss: 1.0171 - val_accuracy: 0.8754
Epoch 61/80
48000/48000 [==============================] - 6s 126us/sample - loss: 0.4054 - ac

In [ ]:
dfTest = pd.read_csv('testX.csv')
count = 10
while count < 100:
    X_train, y_train, X_val, y_val = data()
    model = Sequential([
        Conv2D(32, kernel_size=3, activation='relu',padding='same', input_shape=(28,28,1)),
        Conv2D(32, kernel_size=3, activation='relu',padding='same'),
        MaxPool2D(pool_size=2,strides=2),
        Dropout(0.13243678),
        Conv2D(64, kernel_size=3, activation='relu'),
        Conv2D(64, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPool2D(pool_size=2,strides=2),
        Dropout(0.25767502),
        Flatten(),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.31440486),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.12715375),
        Dense(5, activation='softmax')
    ])
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(lr=0.001))
    model.fit(X_train, y_train, batch_size=128, epochs=40, verbose=0, validation_data=(X_val, y_val), callbacks = [reduce_lr])
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    if(acc >= 0.9):
        print('Val accuracy:', acc)
        gen_csv(model, 'CNN1' + str(count))
        count += 1

Val accuracy: 0.90575
Val accuracy: 0.90575
Val accuracy: 0.9015
Val accuracy: 0.90566665
Val accuracy: 0.90775
Val accuracy: 0.90758336
Val accuracy: 0.90816665
Val accuracy: 0.9091667
Val accuracy: 0.91125
Val accuracy: 0.9008333
Val accuracy: 0.9058333
Val accuracy: 0.9073333
Val accuracy: 0.9045
Val accuracy: 0.9134167
Val accuracy: 0.9088333
Val accuracy: 0.9069167
Val accuracy: 0.9116667
Val accuracy: 0.9061667
Val accuracy: 0.90533334
Val accuracy: 0.9035
Val accuracy: 0.91116667
Val accuracy: 0.90175
Val accuracy: 0.9095
Val accuracy: 0.90316665
Val accuracy: 0.90433335
Val accuracy: 0.9061667
Val accuracy: 0.91116667
Val accuracy: 0.906
Val accuracy: 0.90783334
Val accuracy: 0.90358335
Val accuracy: 0.9005833
Val accuracy: 0.91008335
Val accuracy: 0.90416664
Val accuracy: 0.90208334
Val accuracy: 0.91358334
Val accuracy: 0.9066667
Val accuracy: 0.90741664
Val accuracy: 0.90008336
Val accuracy: 0.9095
Val accuracy: 0.90758336
Val accuracy: 0.9066667
Val accuracy: 0.9023333
Val 

In [3]:
dfTest = pd.read_csv('testX.csv')
count = 1
while count < 30:
    X_train, y_train, X_val, y_val = data()
    model = Sequential([
        Conv2D(32, kernel_size=3, activation='relu',padding='same', input_shape=(28,28,1)),
        Conv2D(32, kernel_size=3, activation='relu',padding='same'),
        MaxPool2D(pool_size=2,strides=2),
        Dropout(0.13243678),
        Conv2D(64, kernel_size=3, activation='relu'),
        Conv2D(64, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPool2D(pool_size=2,strides=2),
        Dropout(0.25767502),
        Flatten(),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.31440486),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.12715375),
        Dense(5, activation='softmax')
    ])
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(lr=0.001))
    model.fit(X_train, y_train, batch_size=128, epochs=40, verbose=0, validation_data=(X_val, y_val), callbacks = [reduce_lr])
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    if(acc >= 0.91):
        print('Val accuracy:', acc)
        gen_csv(model, 'CNN91' + str(count))
        count += 1

Val accuracy: 0.91116667
Val accuracy: 0.91141665
Val accuracy: 0.91025
Val accuracy: 0.91066664
Val accuracy: 0.91025
Val accuracy: 0.9101667
Val accuracy: 0.91183335
Val accuracy: 0.9109167
Val accuracy: 0.91116667
Val accuracy: 0.91041666
Val accuracy: 0.9105833
Val accuracy: 0.9159167
Val accuracy: 0.91041666
Val accuracy: 0.9105
Val accuracy: 0.91008335
Val accuracy: 0.911


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/jtk/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-bbe6d9d8c63e>", line 26, in <module>
    model.fit(X_train, y_train, batch_size=128, epochs=40, verbose=0, validation_data=(X_val, y_val), callbacks = [reduce_lr])
  File "/home/jtk/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training.py", line 819, in fit
    use_multiprocessing=use_multiprocessing)
  File "/home/jtk/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_v2.py", line 395, in fit
    total_epochs=1)
  File "/home/jtk/anaconda3/envs/tf_gpu/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_v2.py", line 128, in run_one_epoch
    batch_outs = execution_function(iterator)
  File "/home/jtk/anaconda3/envs/tf_gpu/lib/python3.7/site-packa

KeyboardInterrupt: 